In [2]:
import numpy as np
import scanpy as sc

un_treatment = 'DMSO_48hr'
pert_treatment = 'Tram_48hr'

# GEARS

In [ ]:
import pickle

import gears

In [ ]:
# Read and format for GEARS
adata = sc.read_h5ad('../plots/drugseries/tram_data.h5ad')
# adata = adata[adata.obs['Training']]  # Filter to training
adata.obs['cell_type'] = adata.obs['cell_line']
conditions = {
    un_treatment: 'ctrl',
    pert_treatment: 'MAP2K1+MAP2K2'}
adata = adata[adata.obs['treatment'].isin(list(conditions.keys()))]
adata.obs['condition'] = adata.obs['treatment'].cat.rename_categories(conditions)

# Preprocess
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=True)  # TODO: Maybe readd
pert_data = gears.PertData('../plots/drugseries')
pert_data.new_data_process(dataset_name='tram_gears', adata=adata)
pert_data.load(data_path = '../plots/drugseries/tram_gears')

# Split and get dataloader
adata.obs['split'] = 'train'
set2conditions = dict(adata.obs.groupby('split').agg({'condition': lambda x: x}).condition)
set2conditions = {i: j.unique().tolist() for i,j in set2conditions.items()}
set2conditions['val'] = set2conditions['test'] = set2conditions['val']  # We limit val ourselves
split_fname = '../plots/drugseries/tram_gears/splits/custom.pkl'
pickle.dump(set2conditions, open(split_fname, 'wb'))
# {'train': ['ctrl', 'MAP2K1+MAP2K2'], 'val': ['ctrl', 'MAP2K1+MAP2K2']}
pert_data.prepare_split(split='custom', split_dict_path=split_fname, seed=42)
pert_data.get_dataloader(batch_size=32, test_batch_size=128)

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
100%|██████████| 2/2 [01:03<00:00, 31.80s/it]
Done!
Saving new dataset pyg object at ../plots/drugseries/tram_gears/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Creating dataloaders....
Done!


In [ ]:
# Train model
gears_model = gears.GEARS(pert_data, device='cuda:0')
gears_model.model_initialize(hidden_size=32)
gears_model.train(epochs=20)  # KeyError: 'SKIN_ctrl_1' ??


Found local copy...
Start Training...
Epoch 1 Step 1 Train Loss: 0.3667
Epoch 1 Step 51 Train Loss: 0.4371
Epoch 1 Step 101 Train Loss: 0.4648
Epoch 1: Train Overall MSE: 204.3902 Validation Overall MSE: 204.3886. 
Train Top 20 DE MSE: 29814.7441 Validation Top 20 DE MSE: 29814.4883. 
Epoch 2 Step 1 Train Loss: 0.5038
Epoch 2 Step 51 Train Loss: 0.4167
Epoch 2 Step 101 Train Loss: 0.3986
Epoch 2: Train Overall MSE: 31.4396 Validation Overall MSE: 31.4395. 
Train Top 20 DE MSE: 3914.1646 Validation Top 20 DE MSE: 3914.3132. 
Epoch 3 Step 1 Train Loss: 0.4522
Epoch 3 Step 51 Train Loss: 0.4526
Epoch 3 Step 101 Train Loss: 0.3964
Epoch 3: Train Overall MSE: 1.3181 Validation Overall MSE: 1.3181. 
Train Top 20 DE MSE: 189.0000 Validation Top 20 DE MSE: 188.9983. 
Epoch 4 Step 1 Train Loss: 0.5512
Epoch 4 Step 51 Train Loss: 0.3640
Epoch 4 Step 101 Train Loss: 0.4906
Epoch 4: Train Overall MSE: 0.0038 Validation Overall MSE: 0.0038. 
Train Top 20 DE MSE: 0.7020 Validation Top 20 DE MSE: 0.7

KeyError: 'SKIN_ctrl_1'

In [81]:
# Save model
model_fname = '../plots/drugseries/tram_gears/model'
gears_model.save_model(model_fname)
gears_model.load_pretrained(model_fname)

In [86]:
# Predict perturbation
pert = gears_model.predict([['MAP2K1', 'MAP2K2']])['MAP2K1_MAP2K2']
np.save('../plots/drugseries/GEARS_perturbation.npy', pert)

# CPA

In [ ]:
import cpa
import pandas as pd

Global seed set to 0


In [7]:
# Read and format
adata = sc.read_h5ad('../plots/drugseries/tram_data.h5ad')
adata.obs['split'] = 'val'
adata.obs.loc[adata.obs['Training'], 'split'] = 'train'
# # adata = adata[adata.obs['Training']]  # Filter to training
# adata.obs['cell_type'] = adata.obs['cell_line']
# conditions = {
#     un_treatment: 'ctrl',
#     pert_treatment: 'MAP2K1+MAP2K2'}
# adata = adata[adata.obs['treatment'].isin(list(conditions.keys()))]
# adata.obs['condition'] = adata.obs['treatment'].cat.rename_categories(conditions)

In [ ]:
# Prepare data
cpa.CPA.setup_anndata(
    adata,
    perturbation_key='treatment',
    dosage_key=None,
    control_group='DMSO_48hr',
    batch_key=None,
    is_count_data=True,
    categorical_covariate_keys=['cell_line'],
    # deg_uns_key='rank_genes_groups_cov',
    # deg_uns_cat_key='cov_drug_dose',
    max_comb_len=2)


100%|██████████| 13713/13713 [00:00<00:00, 976825.98it/s]


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [ ]:
# Prepare and train model
model = cpa.CPA(
    adata=adata,
    split_key='split',
    train_split='train',
    valid_split='val',
    test_split='val',
    n_latent=32)
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path='../plots/drugseries/cpa/')
# model.save('../plots/drugseries/cpa/', overwrite=True)

100%|██████████| 11/11 [00:00<00:00, 141.33it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:23<11:21:38, 20.49s/it, v_num=1, recon=1.62e+4, r2_mean=0.519, adv_loss=3.36, acc_pert=0.13, acc_cell_line=0.833]   


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [03:15<11:34:36, 20.93s/it, v_num=1, recon=1.51e+4, r2_mean=0.688, adv_loss=2.59, acc_pert=0.161, acc_cell_line=0.945, val_recon=1.63e+4, disnt_basal=0.318, disnt_after=0.363, val_r2_mean=0.55, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.


...


disnt_basal = 0.218819266164188
disnt_after = 0.3618891991244186
val_r2_mean = 0.8362826852327498
val_r2_var = 0.14101181146271577
Epoch 310/2000:  15%|█▌        | 309/2000 [1:56:43<9:52:13, 21.01s/it, v_num=1, recon=1.28e+4, r2_mean=0.846, adv_loss=2.01, acc_pert=0.364, acc_cell_line=0.879, val_recon=1.3e+4, disnt_basal=0.216, disnt_after=0.357, val_r2_mean=0.833, val_KL=nan] 
disnt_basal = 0.2152914383799755
disnt_after = 0.3586681730070986
val_r2_mean = 0.8346480092813999
val_r2_var = 0.13362810875435904
Epoch 320/2000:  16%|█▌        | 319/2000 [2:00:25<9:48:54, 21.02s/it, v_num=1, recon=1.28e+4, r2_mean=0.845, adv_loss=2, acc_pert=0.373, acc_cell_line=0.881, val_recon=1.3e+4, disnt_basal=0.218, disnt_after=0.36, val_r2_mean=0.834, val_KL=nan]     
disnt_basal = 0.21864591965633112
disnt_after = 0.35858445922786936
val_r2_mean = 0.8359903254789725
val_r2_var = 0.13411811644879565
Epoch 330/2000:  16%|█▋        | 329/2000 [2:04:07<9:44:48, 21.00s/it, v_num=1, recon=1.28e+4, r2_mean


Epoch 00349: cpa_metric reached. Module best state updated.



disnt_basal = 0.21635082235900704
disnt_after = 0.3607681201231233
val_r2_mean = 0.8351466893761739
val_r2_var = 0.14297566893943772
Epoch 360/2000:  18%|█▊        | 359/2000 [2:15:13<9:35:07, 21.03s/it, v_num=1, recon=1.28e+4, r2_mean=0.846, adv_loss=1.99, acc_pert=0.383, acc_cell_line=0.876, val_recon=1.3e+4, disnt_basal=0.213, disnt_after=0.359, val_r2_mean=0.834, val_KL=nan] 
disnt_basal = 0.21580566240875193
disnt_after = 0.3595484005878319
val_r2_mean = 0.835434922258568
val_r2_var = 0.1407967806167381
Epoch 370/2000:  18%|█▊        | 369/2000 [2:18:55<9:31:51, 21.04s/it, v_num=1, recon=1.28e+4, r2_mean=0.846, adv_loss=1.98, acc_pert=0.376, acc_cell_line=0.879, val_recon=1.3e+4, disnt_basal=0.216, disnt_after=0.359, val_r2_mean=0.831, val_KL=nan] 
disnt_basal = 0.2148614647691582
disnt_after = 0.35769189669427204
val_r2_mean = 0.832197070983762
val_r2_var = 0.1407263039807501
Epoch 380/2000:  19%|█▉        | 379/2000 [2:22:37<9:26:52, 20.98s/it, v_num=1, recon=1.27e+4, r2_mean=0


Epoch 00389: cpa_metric reached. Module best state updated.



disnt_basal = 0.21161492981009056
disnt_after = 0.36013156306252914
val_r2_mean = 0.8362216434337031
val_r2_var = 0.13843472902494788
Epoch 400/2000:  20%|█▉        | 399/2000 [2:30:02<9:20:53, 21.02s/it, v_num=1, recon=1.27e+4, r2_mean=0.847, adv_loss=2, acc_pert=0.383, acc_cell_line=0.875, val_recon=1.29e+4, disnt_basal=0.216, disnt_after=0.36, val_r2_mean=0.832, val_KL=nan]    
disnt_basal = 0.21591535706673035
disnt_after = 0.3590443061287641
val_r2_mean = 0.8331812524433979
val_r2_var = 0.13468087993301575
Epoch 410/2000:  20%|██        | 409/2000 [2:33:44<9:17:46, 21.03s/it, v_num=1, recon=1.27e+4, r2_mean=0.846, adv_loss=1.97, acc_pert=0.376, acc_cell_line=0.884, val_recon=1.29e+4, disnt_basal=0.218, disnt_after=0.36, val_r2_mean=0.835, val_KL=nan]  
disnt_basal = 0.2169181369259394
disnt_after = 0.3575956032512808
val_r2_mean = 0.8328639419679499
val_r2_var = 0.1358556258286511
Epoch 420/2000:  21%|██        | 419/2000 [2:37:26<9:14:37, 21.05s/it, v_num=1, recon=1.27e+4, r2_me

In [43]:
# Predict perturbations
model.predict(adata, batch_size=1024)

100%|██████████| 14/14 [00:08<00:00,  1.71it/s]


In [58]:
# Prepare data
adata_pert = adata[adata.obs['treatment']==un_treatment].copy()
adata_pert.obs['treatment'] = pert_treatment
cpa.CPA.setup_anndata(
    adata_pert,
    perturbation_key='treatment',
    dosage_key=None,
    control_group='DMSO_48hr',
    batch_key=None,
    is_count_data=True,
    categorical_covariate_keys=['cell_line'],
    # deg_uns_key='rank_genes_groups_cov',
    # deg_uns_cat_key='cov_drug_dose',
    max_comb_len=2)


100%|██████████| 2256/2256 [00:00<00:00, 963972.07it/s]

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [ ]:
# Filter to val
common_mask = np.ones(adata_pert.shape[0], dtype=bool)
# common_mask = ~adata_pert.obs['Training']

# Get CPA output
pred = adata_pert.obsm['CPA_pred'].copy()
np.save('../plots/drugseries/CPA_perturbation.npy', pred[common_mask].mean(axis=0))
df = pd.DataFrame(pred, columns=adata_pert.var['gene_name'])
df['cell_line'] = adata_pert[common_mask].obs['cell_line'].to_numpy()
for cell_line, row in df.groupby('cell_line').mean().iterrows():
    np.save(f'../plots/drugseries/CPA_{cell_line}_perturbation.npy', row.to_numpy())